# Tracking Data Gantt Chart

## Setup

### Ecoscope

In [ ]:
ECOSCOPE_RAW = "https://raw.githubusercontent.com/wildlife-dynamics/ecoscope/master"

!pip install 'ecoscope[analysis,mapping,plotting] @ git+https://github.com/wildlife-dynamics/ecoscope@v1.8.5' &> /dev/null

In [ ]:
import os
import sys

import geopandas as gpd
import pandas as pd

import ecoscope
import ecoscope.plotting as plotting

ecoscope.init()

### Google Drive Setup

In [ ]:
output_dir = "Ecoscope-Outputs"

if "google.colab" in sys.modules:
    from google.colab import drive

    drive.mount("/content/drive/", force_remount=True)
    output_dir = os.path.join("/content/drive/MyDrive/", output_dir)

os.makedirs(output_dir, exist_ok=True)

## Trajectory

In [ ]:
# Open CSV as Pandas DataFrame
ecoscope.io.download_file(
    f"{ECOSCOPE_RAW}/tests/sample_data/vector/er_relocs.csv.zip",
    os.path.join(output_dir, "er_relocs.csv.zip"),
)

data = pd.read_csv(os.path.join(output_dir, "er_relocs.csv.zip"), header=0, index_col=0)

# Create GeoPandas DataFrame from Pandas DataFrame, parsing the geometry column
gdf = gpd.GeoDataFrame(data, geometry=gpd.GeoSeries.from_wkt(data["geometry"], crs=4326))


# Create Relocations from the GeoPandas DataFrame
relocs = ecoscope.base.Relocations.from_gdf(gdf)

# Create Trajectory from Relocations
traj = ecoscope.base.Trajectory.from_relocations(relocs)

## EcoPlot Example

Calculate the number of trajectory segments by month by subject

In [ ]:
df = (
    traj[["segment_start", "extra__subject__name"]]
    .groupby([traj["extra__subject__name"], pd.Grouper(key="segment_start", freq="MS")])
    .apply(len)
    .rename("count")
    .reset_index()
)

Create a dict for styles for each groupby val

In [ ]:
# https://plotly.com/python-api-reference/generated/plotly.graph_objects.Scatter.html

line_style = (
    traj.groupby(["extra__subject__name"])["extra__subject__hex"]
    .first()
    .apply(lambda x: {"line": {"color": x}})
    .to_dict()
)
line_style

Use EcoPlot to create a multi-panel graph (one panel per subject) displaying the monthly data counts calculated previously and using the defined line style

In [ ]:
grouped = df.groupby("extra__subject__name")
epdata = plotting.plot.EcoPlotData(
    grouped,
    "segment_start",
    "count",
    groupby_style=line_style,
)

fig = plotting.plot.ecoplot(
    data=[epdata],
    title="Tracking Data Gantt Chart",
    out_path=os.path.join(output_dir, "tracking_data_gantt_chart.png"),
    # subplot_height=200,
    # subplot_width=1200,
    # vertical_spacing=0.1,
    annotate_name_pos=(0.99, 0.1),
    shared_yaxes="all",
    y_title="Monthly Data Count",
    x_title="Time",
    layout_kwargs={
        "width": 1200,
        "showlegend": True,
    },
)

fig